<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
TIME SERIES DATA PROCESSING
<br>
SCALED INFLOWS
</div>
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
Main Notebook
<br>
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Each part of the following script was used to proccess the raw data for the Scalled Inflows Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
    Importing needed libraries
</div>

In [1]:
import pandas as pd
import re
import csv
import os
import requests
from urllib.parse import urlparse, parse_qs
import datetime
from bs4 import BeautifulSoup
import http.client
from multiprocessing import Pool
import shutil
from datetime import datetime

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    3. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
Indicate the year of all data is referring to in the variable data_year.
</div>

In [3]:
# Year to which data refers to:
data_year = 2016

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Defining the countries and its acronyms modeled in Dispa-SET
</div>

In [4]:
# Define lists of countries and standard times
countries = [
    "Austria", "Belgium", "Bulgaria", "Switzerland", "Cyprus", "Czech Republic",
    "Germany", "Denmark", "Estonia", "Greece", "Spain", "Finland", "France",
    "Croatia", "Hungary", "Ireland", "Italy", "Lithuania", "Luxembourg", "Latvia",
    "Malta", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Sweden",
    "Slovenia", "Slovakia", "United Kingdom"
]

dispaSET_codes = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "EL", "ES", "FI", "FR", "HR", "HU", 
                  "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"
]

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
4. Scaled Inflows and Raw Data Directories Definition
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Defining the path to the folders that are going to content all the data realted to the scaled inflow time series.
</div>

In [5]:
# Additional string to be appended
additional_path = "/RawData/HydroData/ScaledInflows/"
additional_path_1 = "/RawData/HydroData/ScaledInflows/Levels"
additional_path_2 = "/RawData/HydroData/ScaledInflows/ScaledOutflows"
additional_path_3 = "/RawData/HydroData/ScaledInflows/Total_Hydro_Installed_Capacity_per_Production_Type"
additional_path_4 = "/RawData/HydroData/ScaledInflows/Outflows"

# Construct the standard_time_data_folder_path variable
scaled_inflows_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Availability_Factor_folder_path variable
levels_folder_path = dispaSET_unleash_folder_path + additional_path_1

# Construct the Scaled_OutFlows_folder_path variable
scaled_outflows_folder_path = dispaSET_unleash_folder_path + additional_path_2

# Construct the Raw Data Source variable
total_hydro_installed_capacity_per_production_type_folder_path = dispaSET_unleash_folder_path + additional_path_3

# Construct the outflows_folder_path variable
outflows_folder_path = dispaSET_unleash_folder_path + additional_path_4


print("scaled_inflows_folder_path:", scaled_inflows_folder_path)
print("levels_folder_path:", levels_folder_path)
print("scaled_outflows_folder_path:", scaled_outflows_folder_path)
print("total_hydro_installed_capacity_per_production_type_folder_path:", total_hydro_installed_capacity_per_production_type_folder_path)
print("outflows_folder_path:", outflows_folder_path)

scaled_inflows_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/
levels_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels
scaled_outflows_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/ScaledOutflows
total_hydro_installed_capacity_per_production_type_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Total_Hydro_Installed_Capacity_per_Production_Type
outflows_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    4.1. Back Up Directory
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Saving the original files into a Back up folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since in the next steps of the processing data new subfolders and files are going to be created, the original ones are saved in a back up foldet to return them as its default content ones the process will be finished.
</div>

In [6]:
# Define the paths
additional_path_6 = '/RawData/HydroData/Scaled_Inflows_backup/'

# Construct the backup_folder_path variable
backup_folder_path = dispaSET_unleash_folder_path + additional_path_6

print("backup_folder_path:", backup_folder_path)

# Create a backup of the directory
if os.path.exists(backup_folder_path):
    shutil.rmtree(backup_folder_path)  # Remove any existing backup if necessary

shutil.copytree(scaled_inflows_folder_path, backup_folder_path)

print(f"Backup created at {backup_folder_path}")

backup_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/Scaled_Inflows_backup/
Backup created at /home/ray/Dispa-SET_Unleash/RawData/HydroData/Scaled_Inflows_backup/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    5. Outflows Data
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Getting the value of the outflows.
</div>

In [7]:
def process_scaled_outflows(data_year, dispaSET_codes, scaled_outflows_folder_path, scaled_inflows_folder_path, total_hydro_installed_capacity_per_production_type_folder_path):
    """Process the outflows and multiply by the corresponding capacity values from total hydro installed capacity files."""
    
    # Convert data_year to string to ensure consistency
    data_year_str = str(data_year)
    
    # Step 1: Create the Outflows folder inside scaled_inflows_folder_path and copy the structure from scaled_outflows_folder_path
    outflows_folder_path = os.path.join(scaled_inflows_folder_path, 'Outflows')
    
    if os.path.exists(outflows_folder_path):
        shutil.rmtree(outflows_folder_path)
        
    shutil.copytree(scaled_outflows_folder_path, outflows_folder_path)
    
    # Step 2: Erase all rows from the HPHS and HDAM columns in the copied CSV files, leaving just the headers
    for dispa_code in dispaSET_codes:
        dispa_outflows_folder = os.path.join(outflows_folder_path, dispa_code)
        
        for subfolder in ['1h', '30min', '15min']:
            subfolder_path = os.path.join(dispa_outflows_folder, subfolder)
            
            if os.path.exists(subfolder_path):
                csv_file_path = os.path.join(subfolder_path, f"{data_year_str}.csv")
                
                if os.path.exists(csv_file_path):
                    # Read the CSV file
                    df = pd.read_csv(csv_file_path)
                    
                    # Erase the data from HPHS and HDAM columns
                    df['HPHS'] = ''
                    df['HDAM'] = ''
                    
                    # Save the modified CSV
                    df.to_csv(csv_file_path, index=False)
                    print(f"Processed (erased) file: {csv_file_path}")
                    
    # Step 3: Multiply HDAM and HPHS columns by corresponding capacity values
    for dispa_code in dispaSET_codes:
        # Path to the total hydro installed capacity file
        capacity_file_path = os.path.join(total_hydro_installed_capacity_per_production_type_folder_path, dispa_code, f"{data_year_str}_Total_Hydro_Installed_Capacity_per_Production_Type.csv")
        
        if os.path.exists(capacity_file_path):
            # Read the capacity CSV
            df_capacity = pd.read_csv(capacity_file_path)
            
            # Check if the data_year_str exists as a column in the CSV
            if data_year_str not in df_capacity.columns:
                print(f"Error: Year '{data_year_str}' not found in columns of {capacity_file_path}")
                print(f"Available columns: {df_capacity.columns.tolist()}")
                continue
            
            # Get the Hydro_Water_Reservoir and Hydro_Pumped_Storage capacities
            try:
                hydro_reservoir_value = df_capacity.loc[df_capacity['Production_Type'] == 'Hydro_Water_Reservoir', data_year_str].values[0]
                hydro_pumped_value = df_capacity.loc[df_capacity['Production_Type'] == 'Hydro_Pumped_Storage', data_year_str].values[0]
            except IndexError:
                print(f"Error: Unable to find required Production_Type values in {capacity_file_path}")
                continue
            
            # Path to the corresponding scaled outflows folder
            scaled_outflows_dispa_folder = os.path.join(scaled_outflows_folder_path, dispa_code)
            outflows_dispa_folder = os.path.join(outflows_folder_path, dispa_code)
            
            for subfolder in ['1h', '30min', '15min']:
                subfolder_path = os.path.join(scaled_outflows_dispa_folder, subfolder)
                
                if os.path.exists(subfolder_path):
                    csv_file_path = os.path.join(subfolder_path, f"{data_year_str}.csv")
                    
                    if os.path.exists(csv_file_path):
                        # Read the CSV file
                        df_outflows = pd.read_csv(csv_file_path)
                        
                        # Multiply HDAM by Hydro_Water_Reservoir capacity
                        df_outflows['HDAM'] = df_outflows['HDAM'] * hydro_reservoir_value
                        
                        # Multiply HPHS by Hydro_Pumped_Storage capacity
                        df_outflows['HPHS'] = df_outflows['HPHS'] * hydro_pumped_value
                        
                        # Path to the corresponding file in the Outflows folder
                        outflows_csv_file_path = os.path.join(outflows_dispa_folder, subfolder, f"{data_year_str}.csv")
                        
                        # Save the modified CSV
                        df_outflows.to_csv(outflows_csv_file_path, index=False)
                        print(f"Processed file: {csv_file_path} -> {outflows_csv_file_path}")
        else:
            print(f"Capacity file not found: {capacity_file_path}")

process_scaled_outflows(data_year, dispaSET_codes, scaled_outflows_folder_path, scaled_inflows_folder_path, total_hydro_installed_capacity_per_production_type_folder_path)


Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/AT/1h/2016.csv
Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/AT/30min/2016.csv
Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/AT/15min/2016.csv
Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/BE/1h/2016.csv
Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/BG/1h/2016.csv
Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/CH/1h/2016.csv
Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/CY/1h/2016.csv
Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/CY/30min/2016.csv
Processed (erased) file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Outflows/CZ/1h/2016.csv
Processed 

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    6. Scaled Inflow Files
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Creating the files where the scaled inflows are going to be saved.
</div>

In [8]:
def join_csv_files(data_year, dispaSET_codes, levels_folder_path, outflows_folder_path, scaled_inflows_folder_path):
    # Convert data_year to string to ensure consistency
    data_year_str = str(data_year)
    
    for dispa_code in dispaSET_codes:
        for subfolder in ['1h', '30min', '15min']:
            # Paths to the respective CSV files
            levels_csv_path = os.path.join(levels_folder_path, dispa_code, subfolder, f"{data_year_str}.csv")
            outflows_csv_path = os.path.join(outflows_folder_path, dispa_code, subfolder, f"{data_year_str}.csv")
            
            # Check if both files exist
            if os.path.exists(levels_csv_path) and os.path.exists(outflows_csv_path):
                # Read the CSV files
                df_levels = pd.read_csv(levels_csv_path)
                df_outflows = pd.read_csv(outflows_csv_path)
                
                # Select relevant columns from each dataframe
                df_levels_selected = df_levels[['Unnamed: 0', 'HPHS_MW', 'HDAM_MW']]
                df_outflows_selected = df_outflows[['HPHS', 'HDAM']]
                
                # Merge the dataframes on the 'Unnamed: 0' column
                df_merged = pd.concat([df_levels_selected, df_outflows_selected], axis=1)
                
                # Path to save the new CSV file
                output_dir = os.path.join(scaled_inflows_folder_path, dispa_code, subfolder)
                os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
                
                output_csv_path = os.path.join(output_dir, f"{data_year_str}.csv")
                
                # Save the merged dataframe to the new CSV file
                df_merged.to_csv(output_csv_path, index=False)
                
                print(f"Processed and saved: {output_csv_path}")
            else:
                print(f"Skipping {dispa_code}/{subfolder} as one or both files do not exist.")

join_csv_files(data_year, dispaSET_codes, levels_folder_path, outflows_folder_path, scaled_inflows_folder_path)

Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/30min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/15min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/1h/2016.csv
Skipping BE/30min as one or both files do not exist.
Skipping BE/15min as one or both files do not exist.
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/1h/2016.csv
Skipping BG/30min as one or both files do not exist.
Skipping BG/15min as one or both files do not exist.
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CH/1h/2016.csv
Skipping CH/30min as one or both files do not exist.
Skipping CH/15min as one or both files do not exist.
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/1h/2016.csv
Proc

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Calculating the corresponding inflow.
</div>

In [9]:
def calculate_inflows(df):
    """Calculate HPHS_Inflow and HDAM_Inflow columns."""
    # Initialize the lists for inflows
    hphs_inflow = []
    hdam_inflow = []

    # Extract relevant columns
    hphs_mw = df['HPHS_MW']
    hphs = df['HPHS']
    hdam_mw = df['HDAM_MW']
    hdam = df['HDAM']

    # Calculate inflows for each row
    for i in range(len(df)):
        if i == 0:
            # For the first row, use the equations: first HPHS_Inflow row = Z and first HDAM_Inflow row = C
            hphs_inflow_value = hphs.iloc[i]
            hdam_inflow_value = hdam.iloc[i]
        else:
            # For subsequent rows, use the provided equations
            hphs_inflow_value = (hphs_mw.iloc[i] - hphs_mw.iloc[i - 1]) + hphs.iloc[i]
            hdam_inflow_value = (hdam_mw.iloc[i] - hdam_mw.iloc[i - 1]) + hdam.iloc[i]
        
        hphs_inflow.append(hphs_inflow_value)
        hdam_inflow.append(hdam_inflow_value)

    # Add the new columns to the dataframe
    df['HPHS_Inflow'] = hphs_inflow
    df['HDAM_Inflow'] = hdam_inflow

    return df

def process_csv_files(dispaSET_codes, data_year, scaled_inflows_folder_path):
    """Process each data_year CSV file in the specified folder structure."""
    for code in dispaSET_codes:
        # Construct the folder path for each code
        code_folder_path = os.path.join(scaled_inflows_folder_path, code)
        
        # Iterate over the subfolders (1h, 30min, 15min)
        for subfolder in ['1h', '30min', '15min']:
            subfolder_path = os.path.join(code_folder_path, subfolder)
            
            # Check if the subfolder exists
            if os.path.exists(subfolder_path):
                # Construct the full file path
                file_path = os.path.join(subfolder_path, f'{data_year}.csv')
                
                # Check if the file exists and apply the inflow calculation
                if os.path.exists(file_path):
                    # Read the CSV file
                    df = pd.read_csv(file_path)
                    
                    # Check if required columns are present
                    required_columns = ['HPHS_MW', 'HPHS', 'HDAM_MW', 'HDAM']
                    if all(column in df.columns for column in required_columns):
                        # Calculate inflows
                        df = calculate_inflows(df)
                        
                        # Save the updated dataframe back to the same CSV file
                        df.to_csv(file_path, index=False)
                        
                        print(f"Processed and saved: {file_path}")
                    else:
                        print(f"Missing required columns in file: {file_path}")
                else:
                    print(f"CSV file not found: {file_path}")
            else:
                print(f"Subfolder not found: {subfolder_path}")

process_csv_files(dispaSET_codes, data_year, scaled_inflows_folder_path)

Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/30min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/15min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/1h/2016.csv
Subfolder not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/30min
Subfolder not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/15min
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/1h/2016.csv
Subfolder not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/30min
Subfolder not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/15min
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CH/1h/2016.csv
Subfolder not found: /home/ray/Dispa-SET_Unleash/RawData/H

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Filtering negative values.
</div>

In [10]:
def process_csv_files(dispaSET_codes, scaled_inflows_folder_path, data_year):
    """
    Processes CSV files, replacing negative values with 0.

    Args:
        dispaSET_codes: A list of dispaSET codes.
        scaled_inflows_folder_path: The path to the scaled inflows folder.
        data_year: The data year.
    """

    for code in dispaSET_codes:
        code_folder_path = os.path.join(scaled_inflows_folder_path, code)

        for subfolder in ['1h', '30min', '15min']:
            file_path = os.path.join(code_folder_path, subfolder, f"{data_year}.csv")

            if os.path.exists(file_path):
                df = pd.read_csv(file_path)

                df['HPHS_Inflow'] = df['HPHS_Inflow'].clip(lower=0)
                df['HDAM_Inflow'] = df['HDAM_Inflow'].clip(lower=0)

                df.to_csv(file_path, index=False)
                print(f"Processed and saved: {file_path}")

process_csv_files(dispaSET_codes, scaled_inflows_folder_path, data_year)

Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/30min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/15min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CH/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/30min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CZ/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/DE/1h/2016.csv
Processed and saved: /home/ray/

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Getting the Scaled Inflows values.
</div>

In [11]:
def process_data(dispaSET_codes, total_hydro_installed_capacity_per_production_type_folder_path, scaled_inflows_folder_path, data_year):
    """
    Processes data, multiplies inflows by hydro capacities.

    Args:
        dispaSET_codes: A list of dispaSET codes.
        total_hydro_installed_capacity_per_production_type_folder_path: Path to the total hydro installed capacity folder.
        scaled_inflows_folder_path: Path to the scaled inflows folder.
        data_year: The data year.
    """

    for code in dispaSET_codes:
        capacity_file_path = os.path.join(total_hydro_installed_capacity_per_production_type_folder_path, code, f"{data_year}_Total_Hydro_Installed_Capacity_per_Production_Type.csv")
        capacity_df = pd.read_csv(capacity_file_path)

        hydro_water_reservoir_capacity = capacity_df[capacity_df['Production_Type'] == 'Hydro_Water_Reservoir'][str(data_year)].values[0]
        hydro_pumped_storage_capacity = capacity_df[capacity_df['Production_Type'] == 'Hydro_Pumped_Storage'][str(data_year)].values[0]

        code_folder = os.path.join(scaled_inflows_folder_path, code)

        for subfolder in ['1h', '30min', '15min']:
            file_path = os.path.join(code_folder, subfolder, f"{data_year}.csv")

            if os.path.exists(file_path):
                df = pd.read_csv(file_path)

                if hydro_water_reservoir_capacity != 0:
                    df['HDAM_Inflow'] = df['HDAM_Inflow'] / hydro_water_reservoir_capacity
                else:
                    df['HDAM_Inflow'] = 0

                if hydro_pumped_storage_capacity != 0:
                    df['HPHS_Inflow'] = df['HPHS_Inflow'] / hydro_pumped_storage_capacity
                else:
                    df['HPHS_Inflow'] = 0

                df.to_csv(file_path, index=False)
                print(f"Processed file: {file_path}")

process_data(dispaSET_codes, total_hydro_installed_capacity_per_production_type_folder_path, scaled_inflows_folder_path, data_year)

Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/1h/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/30min/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/15min/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/1h/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/1h/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CH/1h/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/1h/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/30min/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CZ/1h/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/DE/1h/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/DE/30

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Controlling the limits values.
</div>

In [12]:
def process_csv_files(dispaSET_codes, scaled_inflows_folder_path, data_year):
    """
    Processes CSV files, replacing values greater than 1 with 1.

    Args:
        dispaSET_codes: A list of dispaSET codes.
        scaled_inflows_folder_path: The path to the scaled inflows folder.
        data_year: The data year.
    """

    for code in dispaSET_codes:
        code_folder_path = os.path.join(scaled_inflows_folder_path, code)

        for subfolder in ['1h', '30min', '15min']:
            file_path = os.path.join(code_folder_path, subfolder, f"{data_year}.csv")

            if os.path.exists(file_path):
                df = pd.read_csv(file_path)

                df['HPHS_Inflow'] = df['HPHS_Inflow'].clip(upper=1)
                df['HDAM_Inflow'] = df['HDAM_Inflow'].clip(upper=1)

                df.to_csv(file_path, index=False)
                print(f"Processed and saved: {file_path}")

process_csv_files(dispaSET_codes, scaled_inflows_folder_path, data_year)

Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/30min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/15min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CH/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/30min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CZ/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/DE/1h/2016.csv
Processed and saved: /home/ray/

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Getting the final Scaled Inflows Time Series, erasing unnedded columns.
</div>

In [13]:
def process_csv_files(dispaSET_codes, scaled_inflows_folder_path, data_year):
    """
    Processes CSV files, copying inflow values to HPHS and HDAM columns.

    Args:
        dispaSET_codes: A list of dispaSET codes.
        scaled_inflows_folder_path: The path to the scaled inflows folder.
        data_year: The data year.
    """

    for code in dispaSET_codes:
        code_folder_path = os.path.join(scaled_inflows_folder_path, code)

        for subfolder in ['1h', '30min', '15min']:
            file_path = os.path.join(code_folder_path, subfolder, f"{data_year}.csv")

            if os.path.exists(file_path):
                df = pd.read_csv(file_path)

                df['HPHS'] = df['HPHS_Inflow']
                df['HDAM'] = df['HDAM_Inflow']
                df = df.drop(columns=['HPHS_Inflow', 'HDAM_Inflow', 'HPHS_MW', 'HDAM_MW'])

                df.to_csv(file_path, index=False)
                print(f"Processed and saved: {file_path}")

process_csv_files(dispaSET_codes, scaled_inflows_folder_path, data_year)

Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/30min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/15min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CH/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CY/30min/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CZ/1h/2016.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/DE/1h/2016.csv
Processed and saved: /home/ray/

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
At this point of the code the final Scaled Outflows Time Series is for each Country modeled in Dispa-SET is located in the following local directory:
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
'/Local/Path/to/Dispas-SET/RawData/HydroData/ScaledInflows/'
<br>
Inside this path there are folders with the acronym of each country modelled in Dipsa-SET. i.e. AT, BE, CH.... UK
<br>
Inside each of this folders, there are sub folders named by the time stept of the time series. i.e. 1h, 30min and/or 15min.
<br>
Inside these sub folders, it is going to be found the corresponding time series .csv file named with the year of the data. e.g. 2023.csv
</div>

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
9.1 Scaled Inflows Files Final Location
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman; color:skyblue">
Copying the already processed data to the Dispa-SET data base folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 15px; font-family: TimesNewRoman; color:skyblue">
'/Local/Path/home/ray/Dispa-SET_Unleash/Database/HydroData/ScaledInflows'
</div>

In [14]:
additional_path_5 = "/Database/HydroData/ScaledInflows/"

# Construct the power_plants_raw_data_folder_path variable
scaled_inflows_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_5

In [15]:
def copy_csv_files(dispaSET_codes, scaled_inflows_folder_path, scaled_inflows_data_base_folder_path, data_year):
    """
    Copies CSV files from scaled_inflows_folder_path to scaled_inflows_data_base_folder_path.

    Args:
        dispaSET_codes: A list of dispaSET codes.
        scaled_inflows_folder_path: The path to the scaled inflows folder.
        scaled_inflows_data_base_folder_path: The path to the scaled inflows data base folder.
        data_year: The data year.
    """

    for code in dispaSET_codes:
        source_code_folder = os.path.join(scaled_inflows_folder_path, code)
        target_code_folder = os.path.join(scaled_inflows_data_base_folder_path, code)

        if not os.path.exists(target_code_folder):
            os.makedirs(target_code_folder)

        for subfolder in ['1h', '30min', '15min']:
            source_file = os.path.join(source_code_folder, subfolder, f"{data_year}.csv")
            target_file = os.path.join(target_code_folder, subfolder, f"{data_year}.csv")

            if os.path.exists(source_file):
                shutil.copy2(source_file, target_file)
                print(f"Copied {source_file} to {target_file}")

copy_csv_files(dispaSET_codes, scaled_inflows_folder_path, scaled_inflows_data_base_folder_path, data_year)

Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/1h/2016.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ScaledInflows/AT/1h/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/30min/2016.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ScaledInflows/AT/30min/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/AT/15min/2016.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ScaledInflows/AT/15min/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BE/1h/2016.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ScaledInflows/BE/1h/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/BG/1h/2016.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ScaledInflows/BG/1h/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/CH/1h/2016.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ScaledInflows/CH/1h/2016.csv
Copied /home/ray/Dispa

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
10. Scaled Inflows Folder Back Up
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Once all the formating process was done the Scaled Inflows Folder is restored to its defoult state.
</div>

In [16]:
if os.path.exists(scaled_inflows_folder_path):
    shutil.rmtree(scaled_inflows_folder_path)  # Remove the current directory
shutil.copytree(backup_folder_path, scaled_inflows_folder_path)

print(f"Directory restored to original state from {backup_folder_path}")

Directory restored to original state from /home/ray/Dispa-SET_Unleash/RawData/HydroData/Scaled_Inflows_backup/


In [17]:
shutil.rmtree(backup_folder_path)
print(f"Backup folder {backup_folder_path} deleted successfully.")

Backup folder /home/ray/Dispa-SET_Unleash/RawData/HydroData/Scaled_Inflows_backup/ deleted successfully.
